In [1]:
curl -O https://dl.min.io/client/mc/release/linux-amd64/mc
chmod +x mc
mc alias set opensky https://s3.opensky-network.org k4tAd3Qr1h7x8uGn DLGuWfiVn3xyN5CHOC9TANredkW5ifSE
./mc find opensky/ec-datadump/ --path "*/states_2023-*.parquet" | while read -r file; do ./mc cp "$file" data/ec-datadump/; done

In [ ]:
curl -O https://dl.min.io/client/mc/release/linux-amd64/mc
chmod +x mc
mc alias set opensky https://s3.opensky-network.org $OSN_USERNAME $OSN_KEY
./mc find opensky/ec-datadump/ --path "*/states_2023-*.parquet" | while read -r file; do ./mc cp "$file" data/ec-datadump/; done

In [2]:
import pyspark
from pyspark.sql import SparkSession
import pandas as pd
import numpy as np
import os

spark = SparkSession\
    .builder\
    .appName("project_aiu")\
    .config("spark.hadoop.fs.azure.ext.cab.required.group","eur-app-aiu-dev")\
    .config("spark.yarn.access.hadoopFileSystems","abfs://storage-fs@cdpdldev0.dfs.core.windows.net/data/project/aiu.db/unmanaged")\
    .getOrCreate()

directory_path = "data/ec-datadump/"

for file_name in os.listdir(directory_path):
    if file_name.endswith('.parquet'):
        file_path = os.path.join(directory_path, file_name)
        
        # read each parquet file in the directory
        df = pd.read_parquet(file_path)

        # Convert the NumPy array to a list in the 'serials' column
        df['serials'] = df['serials'].apply(lambda arr: arr.tolist() if isinstance(arr, np.ndarray) else arr)

        # Convert the pandas DataFrame to a Spark DataFrame
        spark_df = spark.createDataFrame(df)

        # Write the Spark DataFrame to the Hive table
        spark_df.write.mode("append").insertInto("project_aiu.osn_ec_datadump")

# Stop the SparkSession
spark.stop()


Setting spark.hadoop.yarn.resourcemanager.principal to quinten.goens


/opt/spark/python/lib/pyspark.zip/pyspark/sql/pandas/conversion.py:371: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.


In [ ]:
# Set to use the "project_aiu" database
#spark.sql("show databases;").show()

In [ ]:
# Set to use the "project_aiu" database
#spark.sql("show tables in project_aiu;").show()

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_parquet("data/daily/borealis-osn-3dpi_EGLL_EIDW_2023-01-01 00:00_2023-01-02 00:00.parquet.gz")

In [ ]:
df[df['icao24'] == '4cad7a']

In [ ]:
# Set to use the "project_aiu" database
spark.sql("select * from project_aiu.your_table_name2;").show()

In [ ]:
spark_df = spark.createDataFrame(df)

In [ ]:


# Write the Spark DataFrame to the Hive table
spark_df.write.mode("append").insertInto("your_table_name")

# Stop the SparkSession
spark.stop()

In [ ]:
df

In [ ]:
import pandas as pd

pd.read_parquet("data/daily/borealis-osn-3dpi_EGLL_EIDW_2023-05-17 00:00_2023-05-18 00:00.parquet.gz").to_csv("test.csv")

In [ ]:
from traffic.data.adsb.opensky_impala import Impala
import pandas as pd
from pathlib import Path
from traffic.data import opensky
import itertools
from datetime import datetime, timedelta

cache_dir_path = "/Users/quintengoens/opensky_cache"
cache_dir = Path(cache_dir_path)

def fetch_data(start_time, end_time, ADEP, ADES):
    # Convert to datetime
    start_time = pd.to_datetime(start_time)
    end_time = pd.to_datetime(end_time)
    
    # Generate a list of dates from start_time to end_time
    date_range = pd.date_range(start_time, end_time)
    
    for date in date_range:
        
        # Format date for file naming and for opensky.history input
        date_str = date.strftime('%Y-%m-%d %H:%M')
        
        # Calculate the stop time for this day
        stop_time_str = (date + timedelta(days=1)).strftime('%Y-%m-%d %H:%M')
        
        print()
        print("-"*20)
        print(f"Fetching data for period {date_str} - {stop_time_str}...")
        
        # If the file already exists, do not rerun
        if Path(f"data/daily/borealis-osn-3dpi_{ADEP}_{ADES}_{date_str}_{stop_time_str}.parquet.gz").exists():
            continue
        df_dep = opensky.history(
            start=date_str,
            stop=stop_time_str,
            departure_airport=ADEP,
            arrival_airport=ADES,
            progressbar=False
        )
        
        df_arr = opensky.history(
            start=date_str,
            stop=stop_time_str,
            departure_airport=ADES,
            arrival_airport=ADEP,
            progressbar=False
        )
        
        if pd.isnull(df_dep) and pd.isnull(df_arr):
            df = pd.DataFrame()
            print("Both are NULL")

        if pd.isnull(df_dep) and not pd.isnull(df_arr):
            df = df_arr.data
            print("df_dep is NULL")

        if pd.isnull(df_arr) and not pd.isnull(df_dep):
            df = df_dep.data
            print("df_arr is NULL")
        
        if not pd.isnull(df_arr) and not pd.isnull(df_dep):
            print("both not NULL")
            df = pd.concat([df_dep.data, df_arr.data])
        
        # Convert timestamp column(s) to a lower precision (like microseconds)
        for col in df.columns:
            if pd.api.types.is_datetime64_any_dtype(df[col]):
                df[col] = df[col].values.astype('datetime64[us]')
        
        # Write to compressed parquet file
        df.to_parquet(f"data/daily/borealis-osn-3dpi_{ADEP}_{ADES}_{date_str}_{stop_time_str}.parquet.gz", compression="gzip")
        print("-"*20)
        print()
    return None




In [ ]:
# Apply the function on all the airport combinations possible 
apts = ["EGLL","EIDW", "EKCH", "BIRK", "ESSA"]

# Calculate all the combinations of apt pairs
start_time = "2023-01-01 00:00"
end_time = "2023-06-01 00:00"

for apt1, apt2 in list(itertools.combinations(apts, 2)):
    print(apt1, apt2, datetime.now())
  fetch_data(start_time, end_time, apt1, apt2)
    
    print("Done!")
    print()

